# Segmenting and Clustering Neighborhoods of Toronto

### 1. Fetching data to define neighborhood

To be able to cluster different neighborhoods of the city of Toronto, we will need to define their geospatial locations and boundaries.  We can obtain this data from `geopy`.  But, before we can do that we have to know more about each neigborhood like thier names and postal codes.

We will be able to achieve this by scraping the necessary data from a website.  the information we need is available at `https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M`

In [1]:
#!conda install -c conda-forge beautifulsoup4  #remove leading hashtag if beutifulsoup is not installed.

from bs4 import BeautifulSoup # the beautiful soup library will be used to scrape the data from wikipedia.

#!conda install -c conda-forge lxml # remove leading hashtag if lxml parser is not installed

#!conda install -c conda-forge requests # remove leading hashtag if requests library is not installed
import requests
import csv


In [2]:
#The contents of the webpage are fetched and stored in as the variable 'source'.  'source' is passed into beautiful soup and parsed to return the beautiful soup object.

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

# To work in the portion of the parse tree that we are concerned with, the contents of the table containing the data of interest is stored in the variable 'table'.
table = soup.table.tbody 

# A csv file is created so that the table contents can be written to it.
csv_file = open('toronto_hood.csv', 'w')
csv_writer = csv.writer(csv_file)

# Each row of the table can be looped through and written to the csv file 'toronto_hood.csv'.
for table_row in table.find_all('tr'):
    field_one = table_row.next_element.next_element
    column_one = field_one.text
    
    field_two = field_one.next_sibling.next_sibling
    column_two = field_two.text

    # The /n linending must be removed from the neighbourhood and the 'Not assigned' values be changed to NaN so that pandas can recognize them.
    field_three = field_two.next_sibling.next_sibling
    column_three = field_three.text
    column_three = column_three[:-1]
    
    if column_three == "Not assigned":
        column_three = 'NaN'

    table_row = table_row.next_sibling.next_sibling
    
    csv_writer.writerow([column_one, column_two, column_three])

csv_file.close()

In [3]:
# import libraries

import pandas as pd

In [49]:
# The dataframe is created from the csv.
df = pd.read_csv('toronto_hood.csv')

df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [50]:
# The rows containing NaN are recognized by pandas and can be dropped and the index reset.

df.dropna(axis=0, inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [51]:
# The neighbourhoods are grouped and concantenated into a pandas Series for each postcode and assigned to a variable.
build_list = df.groupby('Postcode')['Neighbourhood'].apply(', '.join)

# A vector containing the distinct postcodes is created so that it can be looped through so the new values for 'Neighbourhood' in 'df'
pc_frame = df['Postcode'].unique()

In [52]:
# By iterating through the the column of Postcodes in 'pc_frame' the concantenated neighbourhoods can be written into the 'Neighbourhood' column of the original dataframe.

for i in range(102):
    n_hood = pc_frame[i]
    df['Neighbourhood'].loc[df['Postcode']== n_hood] = build_list[n_hood]


In [53]:
# As each postcode had one row per each neighbourhood belonging to it, many duplicate rows were created in the for-loop above.  These duplicates are dropped to obtain the final dataframe.
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)
df.shape

df.tail()

,Postcode,Borough,Neighbourhood
97,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
98,M4Y,Downtown Toronto,Church and Wellesley
99,M7Y,East Toronto,Business Reply Mail Processing Centre 969 Eastern
100,M8Y,Etobicoke,"Humber Bay, King's Mill Park, Kingsway Park So..."
101,M8Z,Etobicoke,"Kingsway Park South West, Mimico NW, The Queen..."


### 2. Adding longitude and latitude data to dataframe

In [ ]:
#!conda update -n base -c defaults conda
# !conda install -c conda-forge/label/cf201901 geocoder Proceed[y]
# import geocoder



In [ ]:
# lat_lng_coords = None

# while(lat_lng_coords is None):
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#     lat_lng_coords = g.latlng

# latitude = lat_lng_coords[0]
# longitude = lat_lng_coords[1]

# print(latitude)

In [45]:
path = 'https://cocl.us/Geospatial_data'

lat_long_df = pd.read_csv(path, index_col=False)
lat_long_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
lat_long_df.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [70]:
df.merge(lat_long_df)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
5,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
6,M3B,North York,Don Mills North,43.745906,-79.352188
7,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
8,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
9,M6B,North York,Glencairn,43.709577,-79.445073
